# 論文輪読
那須野薫(Kaoru Nasuno)/ 東京大学松尾研究室(Matsuo Lab., the University of Tokyo)

##メタ情報
タイトル：Learning Phrase Representations using RNN Encoder–Decoder for Statistical Machine Translation  
著者：Cho, K., Van Merriënboer, B., Gulcehre, C., Bahdanau, D., Bougares, F., Schwenk, H., & Bengio, Y.   
公開：2014  
リンク： [arXiv](http://arxiv.org/abs/1406.1078)  
被引用件数：84  

## Abstract
この論文では、2つのRNNからなるRNN Encoder-Decoderという新しいニューラルネットワークを提案する。  
ひとつのRNNがシンボルのシーケンスを固定長のベクトル表現にエンコードし、もうひとつがその表現をシンボルのシーケンスにデコードするというもの。  
提案モデルのエンコーダとデコーダはソースシーケンスをgivenとしたときのターゲットシーケンスの条件付き確率を最大化するようにひとつのネットワークとして学習させる。  
既存のlog-linearモデルの素性にRNN Encoder-Decoderにより算出されたフレーズペアの条件付き確率を利用することで、統計的機械翻訳システムの性能が向上することが実験的に示される。  
定性的には、提案モデルが言語フレーズの意味的・文法的に意味のある表現を学習することを示す。


##選択理由
#### Gated Recurrent Unitが気になっていた。
- 最近RNNをいじっている。
- LSTMよりマトリックス変数が少ないが精度が同程度？らしい。
- 数式の理解と実装が簡単に見える。

#### 可変長データを固定長に直し、さらに可変長にする方法を提案している。

#### 統計的機械翻訳に特別強い興味がある、というわけではない。


## 目次
- RNN
- RNN Encoder-Decoder
- GRU







## RNN
<img src='files/RNN2.png' width="500px"/>
$$h_{<t>} = f(h_{<t-1>}, x_t)$$  

例えば、  
$$h_{<t>} = tanh(W_xx_t + W_hh_{<t-1>})$$  
$W_h, W_x$：パラメタ

In [1]:
import theano
import theano.tensor as T
import numpy
rng = numpy.random.RandomState(1234)
x = T.fmatrix('x')
n_x = 10
n_hidden_e =10
def get_init_weight(n_in, n_out):
    return rng.uniform(
        low=-numpy.sqrt(6. / (n_in + n_out)),
        high=numpy.sqrt(6. / (n_in + n_out)),
        size=(n_in, n_out),
    ).astype('float32')
    
Wh = theano.shared(
    get_init_weight(n_hidden_e, n_hidden_e),
    name='Wh'
)

Wx = theano.shared(
    get_init_weight(n_x, n_hidden_e),
    name='Wx'
)

def step_e(x, h_tm1, Wx, Wh):
    return T.tanh(T.dot(x, Wx) + T.dot(h_tm1, Wh))

h_t_e, _ = theano.scan(
    fn=step_e,
    sequences=x,
    outputs_info=numpy.zeros(n_hidden_e).astype('float32'),
    non_sequences=[Wx, Wh]
)

## RNN Encoder-Decoder
<img src='files/RNN_Encoder-Decoder.png' width="400px"/>
$$論文より引用$$
任意の長さのデータを固定長に変換して、さらに任意の長さに変換するRNN。  
終了記号(=$x_T$)を読み込んだあとのhidden stateが全体の入力のサマリー(=$c$)になっている。  
エンコーダ部分は普通のRNNと同じ。  
デコーダ部分は$y_t$も$h'_{<t>}$も$y_{t-1}$の条件づけられる。  
$$h'_{<t>}=f(h'_{<t-1>}, y_{t-1}, c)$$  
$$y_{t}=f(h'_{<t>}, y_{t-1}, c)$$  
学習では下記の$\theta$を求める。  
$$\max_{\theta}\frac{1}{N}\sum_{n=1}^Nlogp_{\theta}(y_n|x_n)$$

具体例としては、下記のものが考えられる。  
(実際には、GRUと組み合わせているので、下記とは異なる。)
$$c = tanh(Vh_{T})$$  
$$h'_{<t>} = tanh(U_{hh}h'_{<t-1>} + U_{yh}y_{t-1} + U_{ch}c)$$  
$$y_t = tanh(U_{hy}h'_{<t>} + U_{yy}y_{t-1} + U_{cy}c)$$  
$V, U_{hh}, U_{yh}, U_{ch}, U_{hy}, U_{yy}, U_{cy}$：パラメタ

In [2]:
y_len = T.iscalar('y_len')
n_hidden_d = 10
n_c = 10
n_y = 10
V = theano.shared(
    get_init_weight(n_hidden_e, n_c),
    name='V'
)
Uhh = theano.shared(
    get_init_weight(n_hidden_d, n_hidden_d),
    name='Uhh'
)
Uyh = theano.shared(
    get_init_weight(n_y, n_hidden_d),
    name='Uyh'
)
Uch = theano.shared(
     get_init_weight(n_c, n_hidden_d),
    name='Uch'
)

Uhy = theano.shared(
    get_init_weight(n_hidden_d, n_y),
    name='Uhy'
)
Uyy = theano.shared(
    get_init_weight(n_y, n_y),
    name='Uyy'
)
Ucy = theano.shared(
     get_init_weight(n_c, n_y),
    name='Ucy'
)

c = T.dot(h_t_e[-1], V)

def step_d(h_tm1, y_tm1, Uhh, Uyh, Uch, Uhy, Uyy, Ucy, c):
    h_t = T.tanh(T.dot(h_tm1, Uhh) + T.dot(y_tm1, Uyh) + T.dot(c, Uch))
    return [
        h_t,
        T.tanh(T.dot(h_t, Uhy) + T.dot(y_tm1, Uyy) + T.dot(c, Ucy))
    ]

results, _ = theano.scan(
    fn=step_d,
    outputs_info=[numpy.zeros(n_hidden_d).astype('float32'), numpy.zeros(n_y).astype('float32')],
    non_sequences=[Uhh, Uyh, Uch, Uhy, Uyy, Ucy, c],
    n_steps=y_len
)
h_t_d, y = results

## GRU
<img src='files/GRU.png' width="400px"/>
$$論文より引用$$
Gated Recurrent Unit。  
LSTMのように、長期的な表現と短期的な表現を捉える為に提案されたactivation function。  
ただ、LSTMよりもマトリックス変数が少なく、同程度？の性能が出ている。  
なんとなく、こちらの方が良さそうである。  

$r$がreset gate(LSTMでのforget gateのようなもの)で、  
$z$がupdate gate(LSTMでのmemory cellのようなもの)。  
$r$が0に近いと、前のhidden stateを無視して現在のinputのみを考慮する形になる(?)。

$$r_j = \sigma([W_rx]_j + [U_rh_{<t-1>}]_j)$$  
$$z_j = \sigma([W_zx]_j + [U_zh_{<t-1>}]_j)$$  
$$h_j^{t} = z_jh_j^{<t-1>} + (1 - z_j) \tilde{h}_j^{<t>}$$  
$$\tilde{h}_j^{<t>} = tanh([Wx]_j + [U(r \odot h_{<t-1>})]_j)$$  
$W_r, U_r, W_z, U_z, W, U$：パラメタ

実験では、こういったgated unitを使わないとほとんど学習が進まなかったとのこと。

In [3]:
x = T.fmatrix('x')
n_x = 10
n_hidden =10

Wr = theano.shared(
    get_init_weight(n_x, n_hidden),
    name='Wr'
)
Ur = theano.shared(
    get_init_weight(n_hidden, n_hidden),
    name='Ur'
)
Wz = theano.shared(
    get_init_weight(n_x, n_hidden),
    name='Wz'
)
Uz = theano.shared(
    get_init_weight(n_hidden, n_hidden),
    name='Uz'
)
W = theano.shared(
    get_init_weight(n_x, n_hidden),
    name='W'
)
U = theano.shared(
    get_init_weight(n_hidden, n_hidden),
    name='U'
)

def step(x, h_tm1, W, U, Wr, Ur, Wz, Uz):
    r = T.nnet.sigmoid(T.dot(x, Wr) + T.dot(h_tm1, Ur))
    z = T.nnet.sigmoid(T.dot(x, Wz) + T.dot(h_tm1, Uz))
    h_tc = T.tanh(T.dot(x, W) + T.dot(r * h_tm1, U))
    return z * h_tm1 + (1- z) * h_tc

h_t, _ = theano.scan(
    fn=step,
    sequences=x,
    outputs_info=numpy.zeros(n_hidden).astype('float32'),
    non_sequences=[W, U, Wr, Ur, Wz, Uz]
)

## 実験や結論などのそれ以降の内容
統計的機械翻訳に興味ある方は自分で確認してください。  
ざっと見た感じ、いろいろ拡張性あるので、他分野への応用を検討したい方には、お薦めします。

## 発表者の考察
- 動画説明文生成？
- CV遷移生成？

##まちがいがあったら、おしえてください。
